#### Não precisa olhar esse código do Nixxla. Só fiz ele pra mostrar pra vocês os códigos do PredictionManager rodando

In [6]:
# from mlforecast import MLForecast
import pandas as pd

In [ ]:
path = "D:\Programas\PET\M5_full.parquet"

dados = pd.read_parquet(path)

In [ ]:
dados['date'] = pd.to_datetime(dados['date'])

In [ ]:
dados['year_month'] = dados['date'].dt.to_period('M')

In [ ]:
dados['year_month'] = dados['year_month'].dt.to_timestamp()

In [ ]:
df_general = dados.groupby('year_month')['value'].sum().reset_index().set_index('year_month')

In [ ]:
df_hobbies = dados.groupby(['cat_id', 'year_month'])['value'].sum().reset_index().set_index('year_month')
df_hobbies = df_hobbies[df_hobbies['cat_id'] == 'HOBBIES']

In [ ]:
df_nixtla_general = pd.DataFrame({'ds': df_general.index, 'y': df_general['value'].values, 'unique_id' : 'general'})
df_nixtla_hobbies = pd.DataFrame({'ds': df_hobbies.index, 'y': df_hobbies['value'].values, 'unique_id': 'hobbies'})

df_nixtla = pd.concat([df_nixtla_general, df_nixtla_hobbies], ignore_index=True)

In [ ]:
test_size = (int)(df_nixtla.shape[0]*0.2)

df_test = df_nixtla.tail(test_size)
df_train = df_nixtla.drop(df_test.index)    

In [ ]:
from sklearn.base import BaseEstimator

class Naive(BaseEstimator):
    def fit(self, X, y):
        return self

    def predict(self, X):
        return X['lag1']

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression

In [ ]:
selected_models = [
    DecisionTreeRegressor(),
    LinearRegression(),
    Naive()
]

In [ ]:
ml_forecast = MLForecast(
    models = selected_models,
    freq = 'M',
    lags=[1,2,3,4,5,6]
)

In [ ]:
ml_forecast.fit(df_nixtla)

##### Aqui é o que precisa ser salvo

In [ ]:
predict_full = ml_forecast.predict(6)

#### Vocês precisam usar isso aqui pra fazer o dashboard

##### Essa classe serve pra gerenciar as predições no nosso Banco de Dados de predição. Basicamente, serve como uma API pra salvar e consultar as predições feitas pelo Nixxla.

In [7]:
from PredictionsManager import PredictionsManager

##### Esse path pro db eu criei pra testar a classe, então criem onde ficar melhor pra rodar o dashboard

In [2]:
db_path = 'predictions.db'

In [3]:
pred_manager = PredictionsManager(db_path=db_path)

##### Vejam que o meu DataFrame 'pred_full' tem as previsões de todos os modelos que eu selecionei pra cada uma dos unique_id

In [ ]:
predict_full.head(5)

In [ ]:
predict_full.tail(5)

##### Pra salvar essas previsões na base de dados basta passar esse o df_nixtla q a gnt usou pra fazer as previsões e o predict, que o manager vai separar cada previsão em uma tabela com o nome correspondente ao 'unique_id' dela em 'predict_full'. Exemplo: pra o dataframe 'general', o manager vai criar uma tabela 'general' com o df original e uma tabela 'prev_general' com a previsao.

In [ ]:
pred_manager.save_predictions(df_nixtla, predict_full)

##### Pra pegar as previsões, é só usar o método abaixo e pasasr o 'unique_id' correspondente a previsão que você quer e ele vai retornar o dataframe original e o dataframe da previsao. Seguindo o exemplo acima, se vc quiser pegar a previsao pra general, vc vai botar a string 'general' e receber o dataframe original e a previsao dele numa tupla nessa ordem.

In [ ]:
original_series, prev_hobbies = pred_manager.get_prediction('general')
original_series.head()

In [ ]:
prev_hobbies.head()

##### Depois de usar essa classe pra salvar/pegar algum dado no BD, lembrem de fechar a conexão.

In [4]:
pred_manager.close_connection()

Conexão fechada


##### Adicionando previsões

adicionei um arquivo com as previsões dos treinos (no caso, a df_teste)

In [8]:
path_pred = "../../data/predictions.parquet"
path_test = "../../data/tests.parquet"
path_nix = "../../data/all_M5_series.parquet"

df_nix = pd.read_parquet(path_nix)
df_test = pd.read_parquet(path_test)
df_pred = pd.read_parquet(path_pred)

In [9]:
df_nix['ds'] = df_nix['ds'].dt.to_timestamp()

In [11]:
df_nix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1982955 entries, 0 to 1982954
Data columns (total 3 columns):
 #   Column     Dtype         
---  ------     -----         
 0   unique_id  object        
 1   ds         datetime64[ns]
 2   y          int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 45.4+ MB


In [18]:
df_pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 462 entries, 0 to 461
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   ds                462 non-null    datetime64[ns]
 1   AutoARIMA         462 non-null    float32       
 2   ETS               462 non-null    float32       
 3   Theta             462 non-null    float32       
 4   unique_id         462 non-null    object        
 5   LGBMRegressor     462 non-null    float32       
 6   XGBRegressor      462 non-null    float32       
 7   LinearRegression  462 non-null    float32       
dtypes: datetime64[ns](1), float32(6), object(1)
memory usage: 18.2+ KB


In [13]:
df_pred.head()

,ds,AutoARIMA,ETS,Theta,unique_id,LGBMRegressor,XGBRegressor,LinearRegression
0,2016-06-01,9.958907e+05,1164433.125,1092296.000,RTM,1.033812e+06,1035732.000,1136894.0
1,2016-07-01,1.022800e+06,1164426.375,1095954.625,RTM,1.007906e+06,1022319.000,1140518.0
2,2016-08-01,1.030601e+06,1164421.000,1099613.250,RTM,1.057832e+06,1049496.750,1149934.0
3,2016-09-01,1.007602e+06,1164416.625,1103271.875,RTM,1.054494e+06,1118830.625,1300930.0
4,2016-10-01,1.028526e+06,1164413.250,1106930.500,RTM,1.043091e+06,1009637.875,1228774.0


In [17]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 924 entries, 0 to 923
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   ds                924 non-null    datetime64[ns]
 1   AutoARIMA         924 non-null    float32       
 2   ETS               924 non-null    float32       
 3   Theta             924 non-null    float32       
 4   unique_id         924 non-null    object        
 5   LGBMRegressor     924 non-null    float32       
 6   XGBRegressor      924 non-null    float32       
 7   LinearRegression  924 non-null    float32       
dtypes: datetime64[ns](1), float32(6), object(1)
memory usage: 36.2+ KB


In [20]:
df_test.head(14)

,ds,AutoARIMA,ETS,Theta,unique_id,LGBMRegressor,XGBRegressor,LinearRegression
0,2015-06-01,1.159039e+06,1.095347e+06,1.143320e+06,RTM,1.173777e+06,1.213080e+06,1161696.0
1,2015-07-01,1.173883e+06,1.095444e+06,1.147542e+06,RTM,1.129370e+06,1.251465e+06,1113854.0
2,2015-08-01,1.185385e+06,1.095529e+06,1.151764e+06,RTM,1.163184e+06,1.285105e+06,1163578.0
3,2015-09-01,1.151350e+06,1.095604e+06,1.155986e+06,RTM,1.127464e+06,1.289022e+06,1102324.0
4,2015-10-01,1.156566e+06,1.095670e+06,1.160208e+06,RTM,1.126176e+06,1.234733e+06,1094476.0
5,2015-11-01,1.138070e+06,1.095728e+06,1.164431e+06,RTM,1.141042e+06,1.185206e+06,1085862.0
6,2015-12-01,1.124687e+06,1.095779e+06,1.168653e+06,RTM,1.105323e+06,1.193307e+06,1034818.0
7,2016-01-01,1.149465e+06,1.095824e+06,1.172875e+06,RTM,1.139136e+06,1.159040e+06,1060320.0
8,2016-02-01,1.131241e+06,1.095863e+06,1.177098e+06,RTM,1.139145e+06,1.193976e+06,1061762.0
9,2016-03-01,1.162488e+06,1.095897e+06,1.181320e+06,RTM,1.145364e+06,1.254852e+06,1091998.0


In [21]:
db_path = 'predictions.db'

In [22]:
pred_manager = PredictionsManager(db_path=db_path)

In [23]:
pred_manager.save_predictions(df_nix, df_pred, df_test)

Todas as previsões foram salvas em tabelas chamadas: ['RTM' 'CA' 'TX' 'WI' 'CA_1' 'CA_2' 'CA_3' 'CA_4' 'TX_1' 'TX_2' 'TX_3'
 'WI_1' 'WI_2' 'WI_3' 'FOODS' 'HOBBIES' 'HOUSEHOLD'
 'FOODS_3_090_CA_3_evaluation' 'FOODS_3_586_TX_2_evaluation'
 'FOODS_3_586_TX_3_evaluation' 'FOODS_3_586_CA_3_evaluation'
 'FOODS_3_090_CA_1_evaluation' 'FOODS_3_090_WI_3_evaluation'
 'FOODS_3_090_TX_2_evaluation' 'FOODS_3_090_TX_3_evaluation'
 'FOODS_3_252_TX_2_evaluation' 'FOODS_3_586_TX_1_evaluation'
 'HOBBIES_1_234_CA_3_evaluation' 'HOBBIES_1_348_CA_1_evaluation'
 'HOBBIES_1_371_CA_1_evaluation' 'HOBBIES_1_254_CA_3_evaluation'
 'HOBBIES_1_371_CA_3_evaluation' 'HOBBIES_1_348_CA_3_evaluation'
 'HOBBIES_1_268_CA_1_evaluation' 'HOBBIES_1_341_CA_3_evaluation'
 'HOBBIES_1_015_CA_3_evaluation' 'HOBBIES_1_268_CA_3_evaluation'
 'HOUSEHOLD_1_118_CA_3_evaluation' 'HOUSEHOLD_1_459_CA_3_evaluation'
 'HOUSEHOLD_1_334_CA_3_evaluation' 'HOUSEHOLD_1_303_CA_3_evaluation'
 'HOUSEHOLD_1_521_CA_3_evaluation' 'HOUSEHOLD_1_459_WI_2

In [24]:
original_series, prev_ca, test_ca = pred_manager.get_prediction('RTM')
prev_ca.head(10)

,ds,AutoARIMA,ETS,Theta,unique_id,LGBMRegressor,XGBRegressor,LinearRegression
0,2016-06-01,9.958907e+05,1164433.125,1092296.000,RTM,1.033812e+06,1035732.000,1136894.0
1,2016-07-01,1.022800e+06,1164426.375,1095954.625,RTM,1.007906e+06,1022319.000,1140518.0
2,2016-08-01,1.030601e+06,1164421.000,1099613.250,RTM,1.057832e+06,1049496.750,1149934.0
3,2016-09-01,1.007602e+06,1164416.625,1103271.875,RTM,1.054494e+06,1118830.625,1300930.0
4,2016-10-01,1.028526e+06,1164413.250,1106930.500,RTM,1.043091e+06,1009637.875,1228774.0
5,2016-11-01,9.812590e+05,1164410.500,1110589.125,RTM,1.072867e+06,770532.375,1174666.0


In [25]:
test_ca.head(10)

,ds,AutoARIMA,ETS,Theta,unique_id,LGBMRegressor,XGBRegressor,LinearRegression
0,2015-06-01,1159039.000,1095346.625,1143319.50,RTM,1173776.875,1213079.750,1161696.0
1,2015-07-01,1173882.750,1095443.750,1147541.75,RTM,1129370.375,1251464.875,1113854.0
2,2015-08-01,1185384.875,1095529.125,1151764.00,RTM,1163183.500,1285104.875,1163578.0
3,2015-09-01,1151350.125,1095604.125,1155986.25,RTM,1127464.125,1289021.500,1102324.0
4,2015-10-01,1156565.875,1095670.125,1160208.50,RTM,1126176.250,1234732.625,1094476.0
5,2015-11-01,1138070.250,1095728.000,1164430.75,RTM,1141042.125,1185206.375,1085862.0
6,2015-12-01,1124687.250,1095778.750,1168653.00,RTM,1105322.750,1193307.250,1034818.0
7,2016-01-01,1149465.375,1095823.500,1172875.25,RTM,1139135.875,1159040.375,1060320.0
8,2016-02-01,1131240.750,1095862.750,1177097.50,RTM,1139144.625,1193975.750,1061762.0
9,2016-03-01,1162487.625,1095897.125,1181319.75,RTM,1145364.250,1254852.125,1091998.0


In [26]:
pred_manager.close_connection()

Conexão fechada


# Adicionando tabelas categóricas

In [9]:
import pandas as pd
from PredictionsManager import PredictionsManager

In [2]:
path_state = "../../data/state.parquet"
path_store = "../../data/store.parquet"
path_cat = "../../data/cat.parquet"

df_state = pd.read_parquet(path_state)
df_store = pd.read_parquet(path_store)
df_cat = pd.read_parquet(path_cat)

In [10]:
path_foods = "../../data/foods.parquet"
path_hobbies = "../../data/hobbies.parquet"
path_household = "../../data/household.parquet"

df_foods = pd.read_parquet(path_foods)
df_hobbies = pd.read_parquet(path_hobbies)
df_household = pd.read_parquet(path_household)

In [14]:
db_path = 'predictions.db'

In [15]:
pred_manager = PredictionsManager(db_path=db_path)

In [16]:
pred_manager.add_table(df_state, 'tabela_estados')
pred_manager.add_table(df_store, 'tabela_lojas')
pred_manager.add_table(df_cat, 'tabela_categorias')

pred_manager.add_table(df_foods, 'tabela_comidas')
pred_manager.add_table(df_hobbies, 'tabela_hobbies')
pred_manager.add_table(df_household, 'tabela_casa')

Tabela 'tabela_comidas' adicionada com sucesso.
Tabela 'tabela_hobbies' adicionada com sucesso.
Tabela 'tabela_casa' adicionada com sucesso.


In [18]:
estados = pred_manager.get_table('tabela_estados')
estados

,state_id,value
0,CA,29196717
1,TX,19228405
2,WI,18502051


In [19]:
pred_manager.close_connection()

Conexão fechada
